In [6]:
import sys
import os
current_dir = os.getcwd()
external_path = os.path.join(current_dir, '..', 'src', 'pepbenchmark', 'external')
external_path = os.path.abspath(external_path)
print(external_path)
sys.path.append(external_path)
import warnings
warnings.filterwarnings("ignore")

E:\pycharm\PepBenchmark\src\pepbenchmark\external


In [2]:
import dis
from pepbenchmark.single_peptide.singeltask_dataset import SingleTaskDatasetManager
from pepbenchmark.utils.logging import disable_logging,enable_logging
# disable_logging()
enable_logging()
dataset = SingleTaskDatasetManager(
    dataset_name="test",
    official_feature_names=["fasta","label"],
    force_download=True,
)

print(f"Dataset length: {len(dataset)}")
print("Official features:", dataset.official_feature_dict.keys())
dataset.set_user_feature("other_feature", list(range(len(dataset))))
print("User features:", dataset.user_feature_dict.keys())

# Set split indices
dataset.set_official_split_indices(split_type="random_split", fold_seed=1)
print("Split indices:", dataset.get_split_indices())

# Get train, valid, test features
train_features, valid_features, test_features = dataset.get_train_val_test_features(format="pytorch_dataset")
print("Train features:", train_features[0])

INFO:pepbenchmark.metadata:Dataset directory: C:\Users\wangr/.pepbenchmark_cache/data_share/peptide_dataset/processed_2025.6.12v/
[2025-06-30 15:42:42,286][INFO][pepbenchmark] Feature file ====test/fasta.csv=== exists but force_download is True, will re-download
[2025-06-30 15:42:42,287][INFO][pepbenchmark] Downloading ===test/fasta.csv=== from https://raw.githubusercontent.com/ZGCI-AI4S-Pep/peptide_data/main/test/fasta.csv
[2025-06-30 15:42:43,710][INFO][pepbenchmark] Set official feature: fasta successfully
[2025-06-30 15:42:43,712][INFO][pepbenchmark] Feature file ====test/label.csv=== exists but force_download is True, will re-download
[2025-06-30 15:42:43,713][INFO][pepbenchmark] Downloading ===test/label.csv=== from https://raw.githubusercontent.com/ZGCI-AI4S-Pep/peptide_data/main/test/label.csv
[2025-06-30 15:42:44,453][INFO][pepbenchmark] Set official feature: label successfully
[2025-06-30 15:42:44,454][INFO][pepbenchmark] Set user feature: other_feature successfully
[2025-06-

Dataset length: 10
Official features: dict_keys(['fasta', 'label'])
User features: dict_keys(['other_feature'])


[2025-06-30 15:42:45,564][INFO][pepbenchmark] Set official split ===random_split=== with seed ====1=== successfully


Split indices: {'train': [0, 1, 2, 3, 4, 5, 6, 7], 'valid': [8], 'test': [9]}
Train features: {'official_fasta': 'MTLKVLAGNTGTGPVRLL', 'official_label': np.int64(0), 'user_other_feature': 0}


### Featurization

In [5]:
from pepbenchmark.pep_utils.featurizer import PeptideFeaturizer
fp_featurizer1 = PeptideFeaturizer("fasta", "embedding")
fp_featurizer2 = PeptideFeaturizer("fasta", "biln")
result = fp_featurizer2("ALAGGGPCR")
print(result)


[2025-06-30 15:45:48,353][INFO][pepbenchmark] Failed to instantiate Fasta2Embedding. Reason: Fasta2Embedding.__init__() missing 1 required positional argument: 'model'
[2025-06-30 15:45:48,354][INFO][pepbenchmark] Expected parameters (from docstring):
[2025-06-30 15:45:48,355][INFO][pepbenchmark] 
    Convert FASTA sequence to molecular embedding using a pretrained model.
    If `model` is a string, initialize with Transformers; if it is a PyTorch model instance,
    it can be any `torch.nn.Module`. In that case, the model should provide a `tokenizer` attribute
    or have `config.name_or_path` for inference.

    Args:
        model: HuggingFace model identifier (str) or a PyTorch `nn.Module` instance.
        device: Optional device string (e.g., 'cuda', 'cpu'). Defaults to GPU if available else CPU.
    
[2025-06-30 15:45:48,355][INFO][pepbenchmark] Successfully instantiated Fasta2Biln with parameters: {}


A-L-A-G-G-G-P-C-R


### User defined features and split indices

In [8]:
import pandas as pd
from pepbenchmark.single_peptide.singeltask_dataset import SingleTaskDatasetManager
from pepbenchmark.utils.logging import disable_logging,enable_logging
from pepbenchmark.pep_utils.featurizer import PeptideFeaturizer
from pepbenchmark.splitter.random_spliter import RandomSplitter
import warnings
warnings.filterwarnings("ignore")

enable_logging()
dataset = SingleTaskDatasetManager(
    dataset_name="QS_APML",
    official_feature_names=["fasta","label"],
    dataset_dir = "E:\pycharm\peptide_data\QS_APML")
fasta = dataset.get_official_feature("fasta")
helm_converter = PeptideFeaturizer("fasta", "helm")
smi_converter = PeptideFeaturizer("fasta", "smiles")
smiles = smi_converter(fasta)
fp_featurizer = PeptideFeaturizer("smiles", "fingerprint", {"fp_type": "Morgan", "radius": 2})
helm = helm_converter(fasta)
fp = fp_featurizer(smiles)

dataset.set_user_feature("helm", helm)
dataset.set_user_feature("fingerprint", fp)

spliter = RandomSplitter()
split_indices = spliter.get_split_indices(fasta,
                                        n_splits=5,
                                        frac_train=0.8,
                                        frac_valid=0.1,
                                        frac_test=0.1,
                                        seed=42,
                                    )

dataset.set_user_split_indices(split_indices)

# Get train, valid, test features
train_features, valid_features, test_features = dataset.get_train_val_test_features(format="dict")
train_df = pd.DataFrame(train_features)
train_df.head()

[2025-06-30 15:49:34,557][INFO][pepbenchmark] Set official feature: fasta successfully
[2025-06-30 15:49:34,571][INFO][pepbenchmark] Set official feature: label successfully
[2025-06-30 15:49:34,572][INFO][pepbenchmark] Feature fasta already loaded, skipping download
[2025-06-30 15:49:34,573][INFO][pepbenchmark] Successfully instantiated Fasta2Helm with parameters: {}
[2025-06-30 15:49:34,574][INFO][pepbenchmark] Successfully instantiated Fasta2Smiles with parameters: {}
[2025-06-30 15:49:34,853][INFO][pepbenchmark] Successfully instantiated Smiles2FP with parameters: {'fp_type': 'Morgan', 'radius': 2}
[15:49:34] DEPRECATION WARNING: please use MorganGenerator
[15:49:34] DEPRECATION WARNING: please use MorganGenerator
[15:49:34] DEPRECATION WARNING: please use MorganGenerator
[15:49:34] DEPRECATION WARNING: please use MorganGenerator
[15:49:34] DEPRECATION WARNING: please use MorganGenerator
[15:49:34] DEPRECATION WARNING: please use MorganGenerator
[15:49:34] DEPRECATION WARNING: plea

,official_fasta,official_label,user_helm,user_fingerprint
395,EMRLSKFFRDFILQRKK,1,PEPTIDE1{E.M.R.L.S.K.F.F.R.D.F.I.L.Q.R.K.K}$$$...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ..."
78,DILIIVGG,1,PEPTIDE1{D.I.L.I.I.V.G.G}$$$V2.0,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
278,VEYHFLSPYVSPRE,0,PEPTIDE1{V.E.Y.H.F.L.S.P.Y.V.S.P.R.E}$$$V2.0,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
305,APTLWARMI,0,PEPTIDE1{A.P.T.L.W.A.R.M.I}$$$V2.0,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ..."
325,EIRQTHNIFFNFFKRR,1,PEPTIDE1{E.I.R.Q.T.H.N.I.F.F.N.F.F.K.R.R}$$$V2.0,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


### Negative sample, Redundancy filter, Homo-based split

In [2]:
import pandas as pd
from pepbenchmark.single_peptide.singeltask_dataset import SingleTaskDatasetManager
from pepbenchmark.splitter.homo_spliter import MMseqs2Spliter
from pepbenchmark.pep_utils.redundancy import RedundancyFilter
from pepbenchmark.utils.logging import disable_logging,enable_logging
from pepbenchmark.pep_utils.neg_sample import NegSampleFromBioactive
enable_logging()

dataset = SingleTaskDatasetManager(
    dataset_name="QS_APML",
    official_feature_names=["fasta","label"],
    dataset_dir = "E:\pycharm\peptide_data\QS_APML",
    force_download = False)

#redundancy filter
fitter = RedundancyFilter(identity=0.3)
fasta = dataset.get_official_feature("fasta")
label = dataset.get_official_feature("label")
#remain_index = fitter(fasta, label)
#dataset.set_remain_feature(remain_index)


#negative_sampling
neg_sampler = NegSampleFromBioactive(dataset)
neg_seqs = neg_sampler(ratio = 1, seed = 42)
dataset.set_negative_samples(neg_seqs)

# homo-based split
remain_fasta = dataset.get_official_feature("fasta")
spliter = MMseqs2Spliter()
split_indices = spliter.get_split_indices(remain_fasta,
                                        n_splits=5,
                                        frac_train=0.8,
                                        frac_valid=0.1,
                                        frac_test=0.1,
                                        identity=0.25,
                                        seed=42,
                                    )
print(split_indices)
dataset.set_user_split_indices(split_indices)

train_features, valid_features, test_features = dataset.get_train_val_test_features(format="dict")
train_df = pd.DataFrame(train_features)
train_df.head()


INFO:pepbenchmark.metadata:Dataset directory: C:\Users\wangr/.pepbenchmark_cache/data_share/peptide_dataset/processed_2025.6.12v/
[2025-07-02 22:16:38,849][INFO][pepbenchmark] Set official feature: fasta successfully
[2025-07-02 22:16:38,851][INFO][pepbenchmark] Set official feature: label successfully
[2025-07-02 22:16:38,852][INFO][pepbenchmark] Feature fasta already loaded, skipping download
[2025-07-02 22:16:38,853][INFO][pepbenchmark] Feature label already loaded, skipping download
[2025-07-02 22:16:38,854][INFO][pepbenchmark] Feature fasta already loaded, skipping download
[2025-07-02 22:16:38,854][INFO][pepbenchmark] Feature label already loaded, skipping download
[2025-07-02 22:16:38,857][INFO][pepbenchmark] Downloading ===neg pool=== from https://raw.githubusercontent.com/ZGCI-AI4S-Pep/peptide_data/main/multitask_peptidepedia.csv
[2025-07-02 22:16:39,993][ERROR][pepbenchmark] Failed to download negative_pool_path: 404 Client Error: Not Found for url: https://raw.githubusercont

HTTPError: 404 Client Error: Not Found for url: https://raw.githubusercontent.com/ZGCI-AI4S-Pep/peptide_data/main/multitask_peptidepedia.csv